# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [181]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 50841
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3600
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=200 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=200"
DEBUG:cachecontrol.controller:Caching b/c of expires header


2021-03-27 08:27:11.769376


In [182]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [183]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,Yew-3 Tour Stage 1,03-26 19:00,/events/80263,[race],6,4.97 km,34 m,38 m,Newry_Stage_1_Prologue,/courses/156943,False
1,Breakfast Club,03-27 07:00,/events/73542,[groupride],4,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
2,WESTERLEY CC OPEN 10,03-27 08:00,/events/73962,"[race, itt, ranked]",33,16.11 km,70 m,77 m,Gt.Missenden H4-10 R,/courses/27081,True
3,AHP SERIES #4,03-27 09:00,/events/81824,[race],12,80.13 km,1.36 km,1.30 km,AHP CTRD - TRCS,/courses/148468,False
4,Tour de Waffle,03-27 09:00,/events/73442,[race],4,25.16 km,614 m,614 m,Paterberg,/courses/88,False
...,...,...,...,...,...,...,...,...,...,...,...
150,WESTERLEY VR 3.9 TT Race,08-04 18:30,/events/72907,"[race, itt, ranked]",15,15.64 km,43 m,43 m,Waltham St.Lawrence,/courses/137130,True
151,WESTERLEY VR RACE 3.10,08-18 18:30,/events/75315,"[race, ranked]",10,15.64 km,72 m,45 m,Aces High WVRace,/courses/144701,True
152,WESTERLEY VR RACE 3.11,09-01 19:30,/events/72711,"[race, ranked]",14,16.30 km,37 m,8 m,Young Berks And Old,/courses/137898,True
153,WESTERLEY VR RACE 3.12,09-15 18:30,/events/75316,"[race, ranked]",9,16.38 km,119 m,119 m,The Hillingdon Honey Races,/courses/141054,True


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [184]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [185]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-03-26 19:00:00+00:00,Yew-3 Tour Stage 1,03-26 19:00,/events/80263,[race],6,4.97 km,34 m,38 m,Newry_Stage_1_Prologue,/courses/156943,False
2021-03-27 07:00:00+00:00,Breakfast Club,03-27 07:00,/events/73542,[groupride],4,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False
2021-03-27 08:00:00+00:00,WESTERLEY CC OPEN 10,03-27 08:00,/events/73962,"[race, itt, ranked]",33,16.11 km,70 m,77 m,Gt.Missenden H4-10 R,/courses/27081,True
2021-03-27 09:00:00+00:00,AHP SERIES #4,03-27 09:00,/events/81824,[race],12,80.13 km,1.36 km,1.30 km,AHP CTRD - TRCS,/courses/148468,False
2021-03-27 09:00:00+00:00,Tour de Waffle,03-27 09:00,/events/73442,[race],4,25.16 km,614 m,614 m,Paterberg,/courses/88,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [186]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-03-26 19:00:00+00:00 Yew-3 Tour Stage 1 ['race']
2021-03-27 07:00:00+00:00 Breakfast Club ['groupride']
2021-03-27 08:00:00+00:00 WESTERLEY CC OPEN 10 ['race', 'itt', 'ranked']
2021-03-27 09:00:00+00:00 AHP SERIES #4 ['race']
2021-03-27 09:00:00+00:00 Tour de Waffle ['race']
2021-03-27 09:00:00+00:00 Chat Laps ['groupride']
2021-03-27 09:00:00+00:00 Virtuslo Medio Fondo ['race', 'ranked']
2021-03-27 10:00:00+00:00 Chat Laps ['groupride']
2021-03-27 10:00:00+00:00 CTT 25 mile TT ['race', 'itt']
2021-03-27 10:00:00+00:00 Yew-3 Tour stage 2 ['race']
2021-03-27 12:00:00+00:00 Breakfast Club ['groupride']
2021-03-27 12:00:00+00:00 WKG's SDW. Stage 3 ['race']
2021-03-27 12:30:00+00:00 #100 Chilterns Cheat ['groupride']
2021-03-27 13:00:00+00:00 OTR March Monuments 4-2 ['groupride']
2021-03-27 13:00:00+00:00 OTR March Monument Race 4 ['race', 'ranked']
2021-03-27 14:00:00+00:00 Lou's Saturday Group Ride ['groupride']
2021-03-27 14:00:00+00:00 Weekend Warrior ['groupride']
2021-03-27 15:0

## Write to File

Can use to manually import into Google, other calendars

In [187]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [188]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [189]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [190]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-03-26 12:30:00+00:00 - #100 Salcombe Hill ['groupride'] 1/5.68 km/198 m
2021-03-27 12:30:00+00:00 - #100 Chilterns Cheat ['groupride'] 1/39.73 km/407 m
2021-03-27 19:30:00+00:00 - #100 London ['groupride'] 1/22.46 km/203 m
2021-03-28 09:00:00+00:00 - AHP Series #4 ['race'] 4/80.13 km/1.36 km
2021-03-28 11:30:00+00:00 - #100 South Coast  ['groupride'] 1/29.71 km/521 m
2021-03-28 18:30:00+00:00 - #100 Brecon Beacons ['groupride'] 1/31.03 km/934 m
2021-03-25 19:00:00+00:00 - Team3R Group Ride ['groupride'] 2/22.43 km/515 m
2021-03-29 11:30:00+00:00 - #100 Michaelgate ['groupride'] 1/9.89 km/100 m
2021-03-29 18:30:00+00:00 - #100 Challacombe ['groupride'] 1/18.14 km/416 m
2021-03-30 11:30:00+00:00 - #100 Dovers Hill ['groupride'] 1/8.35 km/187 m
2021-03-30 12:30:00+00:00 - Herd Take it Easy Tuesday ['groupride'] 1/20.00 km/114 m
2021-03-30 19:05:00+00:00 - ELV FLAT CRIT UNDER 300W ['race', 'ranked'] 1/40.98 km/207 m
2021-03-30 19:30:00+00:00 - #100 Cheddar Gorge ['groupride'] 1/32.77 

In [191]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-26 19:00:00+00:00,Yew-3 Tour Stage 1,03-26 19:00,/events/80263,[race],6,4.97 km,34 m,38 m,Newry_Stage_1_Prologue,/courses/156943,False,kae20jfb20q2g994hviibjeaf0,2021-03-26 19:00:00+00:00 - Yew-3 Tour Stage 1...,True
2021-03-27 07:00:00+00:00,Breakfast Club,03-27 07:00,/events/73542,[groupride],4,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,2st82277082hs8641ndlmi6r98,2021-03-27 07:00:00+00:00 - Breakfast Club ['g...,True
2021-03-27 08:00:00+00:00,WESTERLEY CC OPEN 10,03-27 08:00,/events/73962,"[race, itt, ranked]",33,16.11 km,70 m,77 m,Gt.Missenden H4-10 R,/courses/27081,True,5lc3m4e8fo40pts41p0lp2ngmc,2021-03-27 08:00:00+00:00 - WESTERLEY CC OPEN ...,True
2021-03-27 09:00:00+00:00,AHP SERIES #4,03-27 09:00,/events/81824,[race],12,80.13 km,1.36 km,1.30 km,AHP CTRD - TRCS,/courses/148468,False,srenr0pmmhm9a11aflb3t1kvc8,2021-03-27 09:00:00+00:00 - AHP SERIES #4 ['ra...,True
2021-03-27 09:00:00+00:00,Tour de Waffle,03-27 09:00,/events/73442,[race],4,25.16 km,614 m,614 m,Paterberg,/courses/88,False,45ua7fimaif83h07m8862kbgac,2021-03-27 09:00:00+00:00 - Tour de Waffle ['r...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-04 18:30:00+00:00,WESTERLEY VR 3.9 TT Race,08-04 18:30,/events/72907,"[race, itt, ranked]",15,15.64 km,43 m,43 m,Waltham St.Lawrence,/courses/137130,True,is5sjqb12b6bkl0s0h7587ncq8,2021-08-04 18:30:00+00:00 - WESTERLEY VR 3.9 T...,True
2021-08-18 18:30:00+00:00,WESTERLEY VR RACE 3.10,08-18 18:30,/events/75315,"[race, ranked]",10,15.64 km,72 m,45 m,Aces High WVRace,/courses/144701,True,b4apcfaqaageutdk687n3c1k4o,2021-08-18 18:30:00+00:00 - WESTERLEY VR RACE ...,True
2021-09-01 19:30:00+00:00,WESTERLEY VR RACE 3.11,09-01 19:30,/events/72711,"[race, ranked]",14,16.30 km,37 m,8 m,Young Berks And Old,/courses/137898,True,a9jtmfubkb77fb6gg3ti2m2t9s,2021-09-01 19:30:00+00:00 - WESTERLEY VR RACE ...,True


In [192]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-28 12:00:00+00:00,OTR March Monuments 4-3,03-28 12:00,/events/81420,[groupride],3,99.88 km,1.97 km,2.27 km,Liege_3,/courses/161415,False,None,None,False


## Add All Events to Google Calendar

In [193]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    evntString = row['detailsUrl'].replace('/events/', '')

    if row['found'] == False:

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], evntString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

-Updating event:  2021-03-26 19:00:00+00:00 Yew-3 Tour Stage 1 ['race'] 80263
-Updating event:  2021-03-27 07:00:00+00:00 Breakfast Club ['groupride'] 73542
-Updating event:  2021-03-27 08:00:00+00:00 WESTERLEY CC OPEN 10 ['race', 'itt', 'ranked'] 73962
-Updating event:  2021-03-27 09:00:00+00:00 AHP SERIES #4 ['race'] 81824
-Updating event:  2021-03-27 09:00:00+00:00 Tour de Waffle ['race'] 73442
-Updating event:  2021-03-27 09:00:00+00:00 Chat Laps ['groupride'] 73636
-Updating event:  2021-03-27 09:00:00+00:00 Virtuslo Medio Fondo ['race', 'ranked'] 49807
-Updating event:  2021-03-27 10:00:00+00:00 Chat Laps ['groupride'] 73605
-Updating event:  2021-03-27 10:00:00+00:00 CTT 25 mile TT ['race', 'itt'] 81935
-Updating event:  2021-03-27 10:00:00+00:00 Yew-3 Tour stage 2 ['race'] 80269
-Updating event:  2021-03-27 12:00:00+00:00 Breakfast Club ['groupride'] 73573
-Updating event:  2021-03-27 12:00:00+00:00 WKG's SDW. Stage 3 ['race'] 61563
-Updating event:  2021-03-27 12:30:00+00:00 #

In [194]:
df.loc[df['signUps'] > 9]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-27 08:00:00+00:00,WESTERLEY CC OPEN 10,03-27 08:00,/events/73962,"[race, itt, ranked]",33,16.11 km,70 m,77 m,Gt.Missenden H4-10 R,/courses/27081,True,5lc3m4e8fo40pts41p0lp2ngmc,2021-03-27 08:00:00+00:00 - WESTERLEY CC OPEN ...,True
2021-03-27 09:00:00+00:00,AHP SERIES #4,03-27 09:00,/events/81824,[race],12,80.13 km,1.36 km,1.30 km,AHP CTRD - TRCS,/courses/148468,False,srenr0pmmhm9a11aflb3t1kvc8,2021-03-27 09:00:00+00:00 - AHP SERIES #4 ['ra...,True
2021-03-27 09:00:00+00:00,Virtuslo Medio Fondo,03-27 09:00,/events/49807,"[race, ranked]",31,99.97 km,1.33 km,1.31 km,MF Kobansko,/courses/94156,True,vbbift2n4keq3jl66aet6sloo8,2021-03-27 09:00:00+00:00 - Virtuslo Medio Fon...,True
2021-03-27 10:00:00+00:00,CTT 25 mile TT,03-27 10:00,/events/81935,"[race, itt]",10,40.43 km,137 m,274 m,CTT R25/3H,/courses/107825,False,csm5imrkoj1atmdnoqshu135jo,2021-03-27 10:00:00+00:00 - CTT 25 mile TT ['r...,True
2021-03-27 12:00:00+00:00,WKG's SDW. Stage 3,03-27 12:00,/events/61563,[race],21,26.96 km,635 m,469 m,SDW Amberley To Devi,/courses/107881,False,0ll0krncaih97otci6k40prlc8,2021-03-27 12:00:00+00:00 - WKG's SDW. Stage 3...,True
2021-03-27 13:00:00+00:00,OTR March Monument Race 4,03-27 13:00,/events/81814,"[race, ranked]",19,99.88 km,1.97 km,2.27 km,Liege_3,/courses/161415,True,unr0kseds7kdsvt2hs3c98aivg,2021-03-27 13:00:00+00:00 - OTR March Monument...,True
2021-03-27 14:00:00+00:00,Lou's Saturday Group Ride,03-27 14:00,/events/81701,[groupride],13,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,1ueq9h81jjar4thdaoqnuhe834,2021-03-27 14:00:00+00:00 - Lou's Saturday Gro...,True
2021-03-27 18:00:00+00:00,WKG's SDW. Stage 3,03-27 18:00,/events/61496,[race],16,26.96 km,635 m,469 m,SDW Amberley To Devi,/courses/107881,False,sgvh2au8bm6u4kvi7choavratg,2021-03-27 18:00:00+00:00 - WKG's SDW. Stage 3...,True
2021-03-28 09:00:00+00:00,TOK: Stage 2,03-28 09:00,/events/79242,[groupride],11,30.90 km,211 m,200 m,ToK Stage 2 - Elie To St Andrews,/courses/157437,False,mofes9kbmkrh4mpfukir63j52c,2021-03-28 09:00:00+00:00 - TOK: Stage 2 ['gro...,True


In [195]:
df.loc[df['name'].str.startswith('Wacky Races')]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
